In [15]:
import numpy as np
import pandas as pd

In [16]:
round_number = 6

In [17]:
# Load the .npy data
roi_data_folder = f'../../roi_data/round_{round_number}/'
data_status_csv = f'../../roi_data/csv/stat_all_data_noabnormal_extend_mean_normalization_pca_round_{round_number}.csv'

# Load the data status
data_status = pd.read_csv(data_status_csv)
data_status_this_round = data_status[data_status['dev_stage'] == round_number]

# Modify initialization for empty filename->data dict to store arrays
ries_frozen_alive = {}
ries_frozen_dead = {}
ries_normal_alive = {}
ries_normal_dead = {}
cf_frozen_alive = {}
cf_frozen_dead = {}
cf_normal_alive = {}
cf_normal_dead = {}
con_frozen_alive = {}
con_frozen_dead = {}
con_normal_alive = {}
con_normal_dead = {}
pn_frozen_alive = {}
pn_frozen_dead = {}
pn_normal_alive = {}
pn_normal_dead = {}

# Load the data and append without flattening
count = 0
for index, row in data_status_this_round.iterrows():
    filename = row['filename']
    cultivar = row['cultivar']
    mortality = row['mortality']
    treatment = row['treatment']
    features = {'pc0': row['pca_0'], 'pc1': row['pca_1'], 'pc2': row['pca_2'], 'rising_waveform': row['rising_waveform_feature'], 'falling_waveform': row['falling_waveform_feature'], 'peak_temperature': row['peak_temperature']}

    video_data = np.load(f'{roi_data_folder}[ROI]{filename}')
    sample_data = { 'video': video_data, 'features': features }

    if cultivar == 'RIES':
        if mortality:
            if treatment == 1:
                ries_normal_alive[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                ries_frozen_alive[filename] = sample_data
            else:
                count += 1
            # (ries_normal_alive if treatment == 1 else ries_frozen_alive).append(sample_data)
        else:
            if treatment == 1:
                ries_normal_dead[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                ries_frozen_dead[filename] = sample_data
            else:
                count += 1
            # (ries_normal_dead if treatment == 1 else ries_frozen_dead).append(sample_data)
    elif cultivar == 'CF':
        if mortality:
            if treatment == 1:
                cf_normal_alive[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                cf_frozen_alive[filename] = sample_data
            else:
                count += 1
            # (cf_normal_alive if treatment == 1 else cf_frozen_alive).append(sample_data)
        else:
            if treatment == 1:
                cf_normal_dead[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                cf_frozen_dead[filename] = sample_data
            else:
                count += 1
            # (cf_normal_dead if treatment == 1 else cf_frozen_dead).append(sample_data)
    elif cultivar == 'CON':
        if mortality:
            if treatment == 1:
                con_normal_alive[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                con_frozen_alive[filename] = sample_data
            else:
                count += 1
            # (con_normal_alive if treatment == 1 else con_frozen_alive).append(sample_data)
        else:
            if treatment == 1:
                con_normal_dead[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                con_frozen_dead[filename] = sample_data
            else:
                count += 1
            # (con_normal_dead if treatment == 1 else con_frozen_dead).append(sample_data)
    elif cultivar == 'PN':
        if mortality:
            if treatment == 1:
                pn_normal_alive[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                pn_frozen_alive[filename] = sample_data
            else:
                count += 1
            # (pn_normal_alive if treatment == 1 else pn_frozen_alive).append(sample_data)
        else:
            if treatment == 1:
                pn_normal_dead[filename] = sample_data
            elif treatment > 1 and treatment < 5:
                pn_frozen_dead[filename] = sample_data
            else:
                count += 1
            # (pn_normal_dead if treatment == 1 else pn_frozen_dead).append(sample_data)
print(f'count:', count)

# split apart the dict, ensure the shape is (x, 600, 21, 21)
def split_dict(sample_dict, shape=(600, 21, 21)):
    video_dict = {}
    features_dict = {}

    for k, v in sample_dict.items():
        if isinstance(v, dict) and 'video' in v and 'features' in v:
            try:
                video_dict[k] = np.array(v['video'])
                features_dict[k] = v['features']
            except Exception as e:
                print(f"Error processing key {k}: {e}")
        else:
            print(f"Skipping invalid entry: key={k}, value={v}")

    return video_dict, features_dict
    
ries_frozen_alive_video, ries_frozen_alive_features = split_dict(ries_frozen_alive)
ries_frozen_dead_video, ries_frozen_dead_features = split_dict(ries_frozen_dead)
ries_normal_alive_video, ries_normal_alive_features = split_dict(ries_normal_alive)
ries_normal_dead_video, ries_normal_dead_features = split_dict(ries_normal_dead)
cf_frozen_alive_video, cf_frozen_alive_features = split_dict(cf_frozen_alive)
cf_frozen_dead_video, cf_frozen_dead_features = split_dict(cf_frozen_dead)
cf_normal_alive_video, cf_normal_alive_features = split_dict(cf_normal_alive)
cf_normal_dead_video, cf_normal_dead_features = split_dict(cf_normal_dead)
con_frozen_alive_video, con_frozen_alive_features = split_dict(con_frozen_alive)
con_frozen_dead_video, con_frozen_dead_features = split_dict(con_frozen_dead)
con_normal_alive_video, con_normal_alive_features = split_dict(con_normal_alive)
con_normal_dead_video, con_normal_dead_features = split_dict(con_normal_dead)
pn_frozen_alive_video, pn_frozen_alive_features = split_dict(pn_frozen_alive)
pn_frozen_dead_video, pn_frozen_dead_features = split_dict(pn_frozen_dead)
pn_normal_alive_video, pn_normal_alive_features = split_dict(pn_normal_alive)
pn_normal_dead_video, pn_normal_dead_features = split_dict(pn_normal_dead)

count: 138


In [18]:
# combine all features
total_features = ries_frozen_alive_features | ries_normal_alive_features | \
    cf_frozen_alive_features | cf_normal_alive_features | \
    con_frozen_alive_features | con_normal_alive_features | \
    pn_frozen_alive_features | pn_normal_alive_features | \
    ries_frozen_dead_features | ries_normal_dead_features | \
    cf_frozen_dead_features | cf_normal_dead_features | \
    con_frozen_dead_features | con_normal_dead_features | \
    pn_frozen_dead_features | pn_normal_dead_features

print(f"Total features: {len(total_features)}")
print(f'what one feature looks like: {list(total_features.values())[0]}')

Total features: 1379
what one feature looks like: {'pc0': -662.2641322645497, 'pc1': 192.6570142413354, 'pc2': -14.66827344666204, 'rising_waveform': 1.1180143333053314, 'falling_waveform': 0.6722146636432323, 'peak_temperature': 301.8638699924395}


In [19]:
np.save(f'../data/features/exp_{round_number-1}_total_data.npy', total_features)